# <center>Class 5<br>Callbacks Part 2</center>

## Opjectives
In this class we will learn:
<ul>
    <li>How to add responsiveness to a dash app</li>
    <li>Learn to use more advanced callbacks</li>
    <li>Some of the problems with dash Input/Output system</li>
</ul>

*These are the main imports needed in any dash web app*<br>
Let's start with the basic imports, any user-defined modules, and any data.

In [1]:
import dash
from dash import html, dcc
import pandas as pd
from dash import Input, Output, State
import plotly.graph_objects as go

# This is only needed to that we can run dash in a notebook
from jupyter_dash import JupyterDash

# get the data -> the data comes from the census population growth, there is a python companion program
meta = pd.read_csv('data/metadata.csv')
data = pd.read_csv('data/data.csv')

print(data.head())
print(meta.head())

   year        name         data
0  2019  DP05_0001E  328239523.0
1  2018  DP05_0001E  327167439.0
2  2017  DP05_0001E  325719178.0
3  2016  DP05_0001E  323127515.0
4  2015  DP05_0001E  321418821.0
           id   level_0      level_1           level_2  \
0  DP05_0001E  Estimate  SEX AND AGE  Total population   
1  DP05_0002E  Estimate  SEX AND AGE  Total population   
2  DP05_0003E  Estimate  SEX AND AGE  Total population   
3  DP05_0004E  Estimate  SEX AND AGE  Total population   
4  DP05_0005E  Estimate  SEX AND AGE  Total population   

                             level_3 level_4 level_5  
0                                NaN     NaN     NaN  
1                               Male     NaN     NaN  
2                             Female     NaN     NaN  
3  Sex ratio (males per 100 females)     NaN     NaN  
4                      Under 5 years     NaN     NaN  


## Census Population App:
This app will be able to generate multiple graphic types displaying information about US. population estimates.<br>
The data has been downloaded from the us census page. <br>
This is a very dynamic app used to process and show census data over the last decade. The variable names have been coded up in a multi-layer naming structure. This structure will be passed to 4 levels of dropdowns so that the user can choose what to plot and how to plot it.

In [2]:
# I want to create a function that returns a go Figure which can later be used to add traces to it
def get_figure(fig_title, xtitle = None):
    fig = go.Figure()
    
    fig.update_layout(
        # this is a function taking multiple kwargs where complex args have to be passed as dictionaries
        title = {
            'text': fig_title,
            'y': 0.95,
            'x': 0.5,
            'font': {'size': 22}
        },
        paper_bgcolor = 'white',
        plot_bgcolor = 'white',
        autosize = False,
        height = 400,
        xaxis = {
            'title': xtitle,
            'showline': True, 
            'linewidth': 1,
            'linecolor': 'black'
        },
        yaxis = {
            'showline': True, 
            'linewidth': 1,
            'linecolor': 'black'
        }
    )
    
    return(fig)

In [ ]:
# This makes all possible level-based dropdown options from the meta file
# this is limited to the first 20 just to make it usefull

def get_options(lst):
    return([{'label': l, 'value': l} for l in lst if len(l) < 20])

def get_levels():
    levels = {}
    for col in meta.columns:
        if col not in ['id', 'level_0']:
            levels[col] = get_options(meta[col].dropna().drop_duplicates().to_list())
    return(levels)

get_levels()

## 1. A simple app design
Let's first design an app that has:
<ul>
    <li>One dropdown for each level on the naming for the data-series</li>
    <li>A graph</li>
</ul>

In [5]:
# First initialize the app
app = JupyterDash(
    __name__,
    prevent_initial_callbacks = True
)

DROPDOWN_OPTIONS = get_levels()

# define the initial state of the app -> app.layout is what the user sees from upon launching the app
app.layout = html.Div(
    [
        html.H3('Sandbox for Dash Apps'),
        html.H4('Part 1: A simple static app design'),
       
        # There are hidden options here
        html.Hr(),
        html.P('Please select the corresponding series to be plotted. The graph will automatically appear once the final available level has been reached.'),
        html.Div(
            
            # let's create all the dropdowns as dynamically as possible
            [
                dcc.Dropdown(
                    options = opt,
                    placeholder = 'Select level {}'.format(name.replace('_', ' ')),
                    disabled = False,
                    searchable = True,
                    multi = False,
                    id = '{}_dd'.format(name),
                    style = {'width': '10em'}
                ) for name, opt in DROPDOWN_OPTIONS.items()
            ], style = {'display': 'flex', 'justify-content': 'space-between'}
        
        ),
        html.Hr(),
        dcc.Graph(figure = get_figure('Custom Census Data Plotter'))
        
    ], style = {'border-style': 'solid', 'padding': '1em'}
)

app.run_server(
    mode = 'inline',
    debug = True,
    port = 8000
)

## 2. Functionality
We need the dropdowns to continuously update based on the selection made.<br>
After making a selection of the first, the second must become enabled, and then the third, etc.<br>
Once a terminal selection has been reached, a graph is going to be plotted.

In [6]:
def check_next(values):
    '''
    here I want to check if the selected level is a terminal selection
    This returns a tupple:
        True if next level has options
        True if the current level is a terminal option
    
    values is a list of [value/None]
    '''
    meta_tmp = meta.copy()
    for level, val in enumerate(values):
        if val:
            meta_tmp = meta_tmp.loc[meta_tmp['level_{}'.format(level + 1)] == val].copy()
        else:
            next_level = True if meta_tmp['level_{}'.format(level + 1)].any() else False
            this_level = not meta_tmp.loc[pd.isna(meta_tmp['level_{}'.format(level + 1)])].empty

            break
    
    return(next_level, this_level)

check_next(['SEX AND AGE', 'Total population', None, None, None])

(True, True)

In [8]:
# First initialize the app
app = JupyterDash(
    __name__,
    prevent_initial_callbacks = True
)

DROPDOWN_OPTIONS = get_levels()

# define the initial state of the app -> app.layout is what the user sees from upon launching the app
app.layout = html.Div(
    [
        html.H3('Sandbox for Dash Apps'),
        html.H4('Part 2: Adding functionality to the dropdowns'),
       
        # There are hidden options here
        html.Hr(),
        html.P('Please select the corresponding series to be plotted. The graph will automatically appear once the final available level has been reached.'),
        html.Div(
            # let's create all the dropdowns as dynamically as possible
            [
                dcc.Dropdown(
                    options = opt,
                    placeholder = 'Select level {}'.format(name.replace('_', ' ')),
                    # now I am making only the very first dropdown enabled
                    disabled = False if name == 'level_1' else True,
                    searchable = True,
                    multi = False,
                    id = '{}_dd'.format(name),
                    style = {'width': '10em'}
                ) for name, opt in DROPDOWN_OPTIONS.items()
            ], style = {'display': 'flex', 'justify-content': 'space-between'}
                ),
        html.Hr(),
        dcc.Graph(figure = get_figure('Custom Census Data Plotter')),
        html.Div(id = 'alert')
        
    ], style = {'border-style': 'solid', 'padding': '1em'}
)

# let's use the same dynamics of get_levels to create this callback with list comprehension
# this needs to sequentially enable/disable the dropdowns
@app.callback(
    [
        Output('alert', 'children'),
    ] + [Output('{}_dd'.format(name), 'disabled') for name in DROPDOWN_OPTIONS.keys()],
    [Input('{}_dd'.format(name), 'value') for name in DROPDOWN_OPTIONS.keys()]
)
def check_stuff(*args):
    '''
    The *args contain each one of the ids listed in DROPDOWN_OPTIONS.keys()
    this can be passed into a dictionary to keep easy track of it, or just access it via positional element
    
    First, we need to figure out which dropdown was pressed. This works like this:
       1. First capture the context of the triggers
       2. If no context, then prevent update -> this is really unlike it but it seems to be best-practice
       3. Add the logic to each event
    '''
    
    ctx = dash.callback_context
    if not ctx.triggered:
        from dash.exceptions import PreventUpdate
        print('hey!')
        raise PreventUpdate
    else:
        dropdown = ctx.triggered[0]['prop_id'].split('.')[0]
    
    print(args)
    print(*args)
    
    # check the dropdown succession
    next, terminal = check_next(args)
        
    # First handle the message
    if terminal:
        alert = 'The selection would result in a plot of {}'.format(args)
    else:
        alert = 'Please chose further options'
        
    # Now, let's figure out the output of the dropdowns. Dropdown holds the value of the pressed id.
    # Can we make this any better?
    if dropdown == 'level_1_dd':
        disabled = [False] + [not next] + [True]*3
    if dropdown == 'level_2_dd':
        disabled = [False]*2 + [not next] + [True]*2
    if dropdown == 'level_3_dd':
        disabled = [False]*3 + [not next] + [True]*1
    if dropdown == 'level_4_dd':
        disabled = [False]*4 + [not next]
    if dropdown == 'level_5_dd':
        disabled = [False]*5
        
    return([alert] + disabled)
    
app.run_server(
    mode = 'inline',
    debug = True,
    port = 8010      # I am changing the port so that the apps run individually
)

/Users/duxueyuan/opt/anaconda3/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



('SEX AND AGE', None, None, None, None)
SEX AND AGE None None None None
('SEX AND AGE', 'Under 5 years', None, None, None)
SEX AND AGE Under 5 years None None None
(None, 'Under 5 years', None, None, None)
None Under 5 years None None None
('RACE', 'Under 5 years', None, None, None)
RACE Under 5 years None None None


**Is this working properly? What else do we need to handle?**

## 3. Plotting the graph
Now that we have the basic functionality of the dropdowns. Let's get the plots in.<br>
Here we:
- write a function that returns the graph.
- fixes the dropdown clearing

In [10]:
# let's add one more thing to the check_next function so that we can get the series name based on the level
def check_next(values):
    '''
    here I want to check if the selected level is a terminal selection
    This returns a tupple:
        True if next level has options
        True if the current level is a terminal option
    '''
    meta_tmp = meta.copy()
    for level, val in enumerate(values):
        if val:
            meta_tmp = meta_tmp.loc[meta_tmp['level_{}'.format(level + 1)] == val].copy()
        else:
            next_level = True if meta_tmp['level_{}'.format(level + 1)].any() else False
            this_level = not meta_tmp.loc[pd.isna(meta_tmp['level_{}'.format(level + 1)])].empty

            break
    
    series_name = None
    if this_level and level < len(values):
        series_name = meta_tmp.loc[pd.isna(meta_tmp['level_{}'.format(level + 1)]), 'id']
    else:
        series_name = meta_tmp['id']
    
    return(next_level, this_level, series_name.squeeze())

check_next(['SEX AND AGE', 'Total population', None, None, None])
    

(True, True, 'DP05_0001E')

In [11]:
def plot_curve(series_name, description, fig):
    this_data = data.loc[data['name'] == series_name].copy()     #in case we need to modify the slice
    x = this_data['year'].to_list()
    y = this_data['data'].to_list()
    
    fig.add_trace(go.Scatter(y = y, x = x, name = description))
    
    return(fig)



# For testing
fig = get_figure('Testing figure')
series_name = 'DP05_0001E'
description = 'SEX AND AGE-Total population'
fig = plot_curve(series_name, description, fig)
fig.show()

### back to the app development

In [13]:
# First initialize the app
app = JupyterDash(
    __name__,
    prevent_initial_callbacks = True
)

DROPDOWN_OPTIONS = get_levels()

# define the initial state of the app -> app.layout is what the user sees from upon launching the app
app.layout = html.Div(
    [
        html.H3('Sandbox for Dash Apps'),
        html.H4('Part 3: Getting a graph'),
       
        # There are hidden options here
        html.Hr(),
        html.P('Please select the corresponding series to be plotted. The graph will automatically appear once the final available level has been reached.'),
        html.Div(
            # let's create all the dropdowns as dynamically as possible
            [
                dcc.Dropdown(
                    options = opt,
                    placeholder = 'Select level {}'.format(name.replace('_', ' ')),
                    disabled = False if name == 'level_1' else True,
                    searchable = True,
                    multi = False,
                    id = '{}_dd'.format(name),
                    style = {'width': '10em'}
                ) for name, opt in DROPDOWN_OPTIONS.items()
            ], style = {'display': 'flex', 'justify-content': 'space-between'}
                ),
        html.Hr(),
        dcc.Graph(figure = get_figure('Custom Census Data Plotter'), id = 'figure'),
        html.Div(id = 'alert')
        
    ], style = {'border-style': 'solid', 'padding': '1em'}
)

# let's use the same dynamics of get_levels to create this callback with list comprehension
@app.callback(
    [
        Output('alert', 'children'),
    ] + [Output('{}_dd'.format(name), 'disabled') for name in DROPDOWN_OPTIONS.keys()] + [
        Output('figure', 'figure'),
    ],
    [Input('{}_dd'.format(name), 'value') for name in DROPDOWN_OPTIONS.keys()],
)
def check_stuff(*args):
    '''
    The *args contain each one of the ids listed in DROPDOWN_OPTIONS.keys()
    this can be passed into a dictionary to keep easy track of it, or just access it via positional element
    
    First, we need to figure out which dropdown was pressed. This works like this:
       1. First capture the context of the triggers
       2. If no context, then prevent update
       3. Add the logic to each event
    '''
    
    ctx = dash.callback_context
    if not ctx.triggered:
        from dash.exceptions import PreventUpdate
        raise PreventUpdate
    else:
        dropdown = ctx.triggered[0]['prop_id'].split('.')[0]
       
    # check the dropdown succession
    next, terminal, series_name = check_next(args)
        
    # First handle the message
    if terminal:
        alert = 'The selection would result in a plot of {}'.format(args)
    else:
        alert = 'Please chose further options'
    
    # Now, let's figure out the output of the dropdowns. Dropdown holds the value of the pressed id.
    
    # First, make None everything after a None
    for pos_none, arg in enumerate(args):
        if not arg:
            args = [arg if pos < pos_none else None for pos, arg in enumerate(args)]
    print('Check if the args {} match the dropdowns'.format(args))
    
    # Can we make this any better?
    if dropdown == 'level_1_dd':
        disabled = [False] + [not next] + [True]*3
    elif dropdown == 'level_2_dd':
        disabled = [False]*2 + [not next] + [True]*2
    elif dropdown == 'level_3_dd':
        disabled = [False]*3 + [not next] + [True]*1
    elif dropdown == 'level_4_dd':
        disabled = [False]*4 + [not next]
    elif dropdown == 'level_5_dd':
        disabled = [False]*5
    
    if isinstance(series_name, str):
        fig = plot_curve(series_name, description, fig = get_figure('A brand new figure'))
    else:
        fig = get_figure('A brand new figure')
    
    return([alert] + disabled + [fig])
    
app.run_server(
    mode = 'inline',
    debug = True,
    port = 8020
)

/Users/duxueyuan/opt/anaconda3/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



Check if the args ['RACE', None, None, None, None] match the dropdowns
Check if the args ['RACE', 'Under 5 years', None, None, None] match the dropdowns
Check if the args ['RACE', None, None, None, None] match the dropdowns
Check if the args ['RACE', '10 to 14 years', None, None, None] match the dropdowns


### Observations:
There are a few things to notice/fix here:
1. The figure is re-created everytime which can be more resource consuming,
2. Recall that plotly figures are objects that have sort of memory, 
3. The State() function allows us to take the current state of any component, or figure object, and use it as needed, 
4. **BUT** we still have a problem, we should have a control to make sure the series is not already added, and
5. We can also add the dropdown values to the outputs so that we can fix the discrepancy after clearing

In [14]:
# First initialize the app
app = JupyterDash(
    __name__,
    prevent_initial_callbacks = True
)

DROPDOWN_OPTIONS = get_levels()

# define the initial state of the app -> app.layout is what the user sees from upon launching the app
app.layout = html.Div(
    [
        html.H3('Sandbox for Dash Apps'),
        html.H4('Part 4: Reusing the graph object via callback State()'),
       
        # There are hidden options here
        html.Hr(),
        html.P('Please select the corresponding series to be plotted. The graph will automatically appear once the final available level has been reached.'),
        html.Div(
            # let's create all the dropdowns as dynamically as possible
            [
                dcc.Dropdown(
                    options = opt,
                    placeholder = 'Select level {}'.format(name.replace('_', ' ')),
                    disabled = False if name == 'level_1' else True,
                    searchable = True,
                    multi = False,
                    id = '{}_dd'.format(name),
                    style = {'width': '10em'}
                ) for name, opt in DROPDOWN_OPTIONS.items()
            ], style = {'display': 'flex', 'justify-content': 'space-between'}
                ),
        html.Hr(),
        dcc.Graph(figure = get_figure('Custom Census Data Plotter'), id = 'figure'),
        html.Div(id = 'alert')
        
    ], style = {'border-style': 'solid', 'padding': '1em'}
)

# let's use the same dynamics of get_levels to create this callback with list comprehension
@app.callback(
    [
        Output('alert', 'children'),
    ] + [
        Output('{}_dd'.format(name), 'disabled') for name in DROPDOWN_OPTIONS.keys()
    ] + [
        Output('{}_dd'.format(name), 'value') for name in DROPDOWN_OPTIONS.keys()
    ] + [
        Output('figure', 'figure'),
    ],
    [Input('{}_dd'.format(name), 'value') for name in DROPDOWN_OPTIONS.keys()],
    State('figure', 'figure')
)
def populate_cities(*args):
    '''
    The *args contain each one of the ids listed in DROPDOWN_OPTIONS.keys()
    this can be passed into a dictionary to keep easy track of it, or just access it via positional element
    
    First, we need to figure out which dropdown was pressed. This works like this:
       1. First capture the context of the triggers
       2. If no context, then prevent update
       3. Add the logic to each event
    '''
    
    ctx = dash.callback_context
    if not ctx.triggered:
        from dash.exceptions import PreventUpdate
        raise PreventUpdate
    else:
        dropdown = ctx.triggered[0]['prop_id'].split('.')[0]
    
    ######## SINCE I ADDED THE STATE, I need to remove it from the args. But the args is a tupple, so I cannot pop it
    print('This is the state of the figure', type(args[-1]))
    fig = go.Figure(args[-1])
    print('And this is the acctual figure', type(fig))
    args = [arg for arg in args[:-1]]
    #######
    
    # check the dropdown succession
    next, terminal, series_name = check_next(args)
        
    # First handle the message
    if terminal:
        alert = 'The selection would result in a plot of {}'.format(args)
    else:
        alert = 'Please chose further options'
    
    #####################################################################
    ####################### at the very least we could loop 
    #####################################################################
    disabled = [False] * 5
    
    # First, make None everything after a None
    for pos_none, arg in enumerate(args):
        if not arg:
            args = [arg if pos < pos_none else None for pos, arg in enumerate(args)]
    
    for x in range(1, len(args) - 1):
        if dropdown == 'level_{}_dd'.format(x):
            # also, add a condition to disable all dropdowns when a clearing happens
            if not args[x-1]:
                disabled = [False]*x + [True]*(len(args) - x)
            else:
                disabled = [False]*x + [not next] + [True]*(len(args) - x - 1)
            break
    
    if isinstance(series_name, str):
        series_label = '-'.join([x.lower() for x in args if x])
        
        # this takes care of the duplication
        add_series = True
        for sct in fig.data:
            if sct.name == series_label:
                add_series = False
        if add_series:        
            fig = plot_curve(series_name, series_label, fig)
    
    return([alert] + disabled + args + [fig])
    
app.run_server(
    mode = 'inline',
    debug = True,
    port = 8030
)

### Final thoughts
Let's do the figure again, but this time let's also add a way to control the figure type

In [15]:
def plot_curve(series_name, description, fig, mode = 'lines'):
    this_data = data.loc[data['name'] == series_name].copy()     #in case we need to modify the slice
    x = this_data['year'].to_list()
    y = this_data['data'].to_list()
    
    fig.add_trace(go.Scatter(y = y, x = x, name = description, mode = mode))
    
    return(fig)

# For testing
fig = get_figure('Testing figure with markers only')
series_name = 'DP05_0001E'
description = 'SEX AND AGE-Total population'
fig = plot_curve(series_name, description, fig, 'markers')
fig.show()
fig = get_figure('Testing figure with lines')
fig = plot_curve(series_name, description, fig)
fig.show()

And now, let's make some sub-plots

In [16]:
from plotly.subplots import make_subplots

# sadly, I have to start up the figure again.
fig = make_subplots(rows = 2, cols = 1)

# Let's grab male and female populations
names = ['Male', 'Female']
for pos, name in enumerate(['DP05_0003E', 'DP05_0002E']):
    fig.add_trace(
        go.Scatter(
            x = data.loc[data['name'] == name]['year'].to_list(),
            y = data.loc[data['name'] == name]['data'].to_list(),
            name = names[pos],
            mode = 'markers',
            ###### This is not shown because of the lines mode
            marker = dict(
                size = [10*x for x in range(10, 1, -1)],
                color = [x for x in range(10)],
            )
        ), row = pos + 1, col = 1
    )
# my usual update
fig.update_layout(
    # this is a function taking multiple kwargs where complex args have to be passed as dictionaries
    title = {
        'text': 'Male and Female US Population',
        'y': 0.95,
        'x': 0.5,
        'font': {'size': 22}
    },
    paper_bgcolor = 'white',
    plot_bgcolor = 'white',
    autosize = False,
    height = 400,
    xaxis = {
        'showline': True, 
        'linewidth': 1,
        'linecolor': 'black'
    },
    xaxis2 = {
        'title': 'Estimated Population',
        'showline': True, 
        'linewidth': 1,
        'linecolor': 'black'
    },
    
    yaxis = {
        'showline': True, 
        'linewidth': 1,
        'linecolor': 'black'
    }
)
